In [ ]:
import cv2
import numpy as np

# Step 1: Get the live video feed from your webcam
cap = cv2.VideoCapture(0)  # Use 0 for the default camera

# Check if the webcam is opened successfully
if not cap.isOpened():
    print("Error: Could not access the webcam.")
    exit()

# Get the width and height of the frames
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Define the codec and create VideoWriter object
out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width, frame_height))

# Initialize background as None
background_frame = None

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # If the frame was captured successfully, ret will be True
    if not ret:
        print("Error: Failed to grab frame.")
        break

    # Convert the current frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Store the first frame as background
    if background_frame is None:
        background_frame = gray_frame
        print("Background frame captured and stored.")
        continue  # Skip the rest of the loop after storing the background
    
    # Compute the absolute difference between the current frame and the background frame
    abs_diff = cv2.absdiff(background_frame, gray_frame)
    
    # Apply threshold to the absolute difference
    _, thresholded = cv2.threshold(abs_diff, 25, 255, cv2.THRESH_BINARY)
    
    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Loop over all contours
    for contour in contours:
        # Check if the contour area is large enough
        if cv2.contourArea(contour) > 500:  # You can adjust this threshold for sensitivity
            # Get the bounding rectangle for the contour
            x, y, w, h = cv2.boundingRect(contour)
            
            # Draw a green rectangle around the detected object
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            
            # Add "UNSAFE" text in red color on the frame
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame, "UNSAFE", (x, y - 10), font, 1, (0, 0, 255), 2, cv2.LINE_AA)
    
    # Write the frame into the file 'output.avi'
    out.write(frame)

    # Display the processed frame with rectangles and text
    cv2.imshow("Motion Detection", frame)

    # Wait for the user to press the 'q' key to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()  # Release the video capture object
out.release()  # Release the video writer object
cv2.destroyAllWindows()  # Close all OpenCV windows


Background frame captured and stored.


: 